In [28]:
from urllib2 import Request, urlopen, URLError
import re
from lxml import html
import requests
from bs4 import BeautifulSoup
import random

#import psycopg2

In [29]:
list_of_cities = ['Alameda','Albany','American Canyon','Antioch','Atherton','Belmont','Belvedere','Benicia','Berkeley','Brentwood','Brisbane','Burlingame','Calistoga','Campbell','Clayton','Cloverdale','Colma','Concord','Corte Madera','Cotati','Cupertino','Daly City','Danville','Dixon','Dublin','East Palo Alto','El Cerrito','Emeryville','Fairfax','Fairfield','Foster City','Fremont','Gilroy','Half Moon Bay','Hayward','Healdsburg','Hercules','Hillsborough','Lafayette','Larkspur','Livermore','Los Altos','Los Altos Hills','Los Gatos','Martinez','Menlo Park','Mill Valley','Millbrae','Milpitas','Monte Sereno','Moraga','Morgan Hill','Mountain View','Napa','Newark','Novato','Oakland','Oakley','Orinda','Pacifica','Palo Alto','Piedmont','Pinole','Pittsburg','Pleasant Hill','Pleasanton','Portola Valley','Redwood City','Richmond','Rio Vista','Rohnert Park','Ross','St. Helena','San Anselmo','San Bruno','San Carlos','San Francisco','San Jose','San Leandro','San Mateo','San Pablo','San Rafael','San Ramon','Santa Clara','Santa Rosa','Saratoga','Sausalito','Sebastopol','Sonoma','South San Francisco','Suisun City','Sunnyvale','Tiburon','Union City','Vacaville','Vallejo','Walnut Creek','Windsor','Woodside','Yountville']
#list_of_cities = ['San Francisco']
list_of_cities = ['San Francisco', 'San Jose']


In [30]:
# GET THE COMPLETE LIST OF BUSINESSES IN THE BAY AREA 

# search terms are : day care, gardener, plumber, house cleaning, auto repair, math tutoring
# must put a '+' where there is a space, e.g. house+cleaning
#DONEtopic="day+care"
#DONEtopic="gardener"
#DONEtopic="plumber"
#DONEtopic="house+cleaning"
#topic="auto+repair"
topic="math+tutoring"

patterns=re.compile(r'="(\S.*?lid.*?)".*?class="business-name"><span itemprop="name">(\S.*?)<')  

mydict = {}
for city in list_of_cities :
    print (city, str(len(mydict)) + " total businesses so far")
    pagenum = 0
    while (1) :
        pagenum = pagenum + 1
        print "--> page " + str(pagenum)
        city = city.replace(" ", "%20")
        requestedsite = "https://www.yellowpages.com/search?search_terms="+topic+ "&geo_location_terms=" + city + "%2C+CA&page=" + str(pagenum)
        request = Request(requestedsite)
        response = urlopen(request)
        groupslist =  response.read().split("\n")
        if (any(re.match(r'.*class="business-name"><span itemprop="name">(\S.*?)<', item) for item in groupslist)) :
            pass
        else :
            break
        for i in groupslist:
            itoks = i.split("href")
            for itokk in itoks :
                matchO = re.match(patterns, itokk)
                if matchO and matchO.group(1) :
                    mydict[matchO.group(2)] = matchO.group(1)
            



('San Francisco', '0 total businesses so far')
--> page 1
--> page 2
--> page 3
--> page 4
--> page 5
--> page 6
--> page 7
--> page 8
--> page 9
--> page 10
--> page 11
--> page 12
--> page 13
--> page 14
--> page 15
--> page 16
--> page 17
--> page 18
--> page 19
--> page 20
--> page 21
--> page 22
('San Jose', '537 total businesses so far')
--> page 1
--> page 2
--> page 3
--> page 4
--> page 5
--> page 6
--> page 7
--> page 8
--> page 9
--> page 10
--> page 11
--> page 12
--> page 13
--> page 14
--> page 15
--> page 16
--> page 17
--> page 18
--> page 19
--> page 20
--> page 21
--> page 22
--> page 23
--> page 24
--> page 25
--> page 26
--> page 27
--> page 28
--> page 29
--> page 30
--> page 31


In [31]:
#EXTRACT EMAILS AND COMPANY WEBSITE

website_dict = {}
email_dict = {}
all = len(mydict)
cnt = 0

for site in mydict :
    cnt = cnt + 1
    company_website = None
    email = None
    subrequestsite = "https://www.yellowpages.com" + mydict[site]
    print str(cnt) + "/" + str(all), site
    request = Request(subrequestsite)
    response = urlopen(request)
    groupslist =  response.read().split("\n")    
    
    for i in groupslist:
                matchO = re.match(r'.*LOC\":\"(.*?)\".*Visit Website', i)
                if matchO and matchO.group(1) :
                    company_website = matchO.group(1)
                if email == None :
                    matchO = re.match(r'.*?mailto:(.*?)\"', i)
                    if matchO and matchO.group(1) :
                        email = matchO.group(1)
    email_dict[site] = email
    website_dict[site] = company_website
                

1/1149 Raising A Reader
2/1149 Sunnyvale School District
3/1149 Melissa's Family Childcare
4/1149 Growing Tree Learning Center
5/1149 Bay Area Head Lice Intervention School Services
6/1149 Pinecone Active Learning
7/1149 Silicon Valley Deca
8/1149 Greatschools Inc
9/1149 Kumon Math and Reading Center of Santa Clara - Agnew
10/1149 American Academy of English
11/1149 Child Education Center
12/1149 Computer History Museum
13/1149 The After School Enrichment Program
14/1149 Learning Accelerator
15/1149 Fidelis Education
16/1149 Costa Rica School Project
17/1149 Amy Child Center
18/1149 Rooftop Elementary
19/1149 Educational Excellence Consulting
20/1149 Be Amazing Learning
21/1149 Heffalump Cooperative Nursery
22/1149 The Reading CLinic
23/1149 Landmark Education
24/1149 LITTLE PEOPLE PRESCHOOL
25/1149 Savvy Success Unlimited
26/1149 Forward Driving School
27/1149 Palo Alto Inst For Research
28/1149 Gallagher Learning Center
29/1149 Californians For Justice Edu
30/1149 Uplift Tutor
31/114

In [32]:
randomized_businesses = mydict.keys()

# keep this seed
seed = 100
random.seed(seed)

random.shuffle(randomized_businesses)

F = open("businesses_" + topic+".txt","w")

for i in randomized_businesses :
    website = "NA"
    email = "NA"
    if i in website_dict :
        website = website_dict[i]
    if i in email_dict :
        email = email_dict[i]
    if website is None :
        website = "NA"
    if email is None :
        email = "NA"
    F.write("\n"+i+"\twww.yellowpages.com"+mydict[i]+"\t"+website+"\t"+email)

F.close()